# 🧪 Waste Detection Project – Data Augmentation

In this notebook, we apply various data augmentation techniques to increase the diversity of our training dataset. These augmentations include brightness adjustment, color shifts, flips, and rotations.

Before we begin, make sure your dataset is structured like this:
```
Garbage-Dataset/
├── images/
│ ├── biological_001.jpeg
│ ├── biological_002.jpeg
│ ├── biological_003.jpeg
│ ├── biological_004.jpeg
│ └── ...
└── labels/
├── biological_001.txt
├── biological_002.txt
├── biological_003.txt
├── biological_004.txt
└── ...
```
We'll apply transformations like flipping, rotation, and brightness changes — while ensuring the bounding box annotations are correctly updated for each transformed image.

## 📚 Import Required Libraries

In [ ]:
import os
import random
from PIL import Image, ImageEnhance, ImageDraw
import matplotlib.pyplot as plt
import math

## 📁 Define Input and Output Paths

Set up the input and output directory paths for images and annotations, including folders to save augmented data.

In [ ]:
# Input folders
input_dir = 'Garbage-Dataset/images'
annotations_dir = 'Garbage-Dataset/labels'

# Output folders
output_dir = "Garbage-Dataset/augmented_images"
augmented_annotations_dir = "Garbage-Dataset/augmented_labels"

## ✅ Ensure Required Directories Exist

In [ ]:
# Check if input and annotation directories exist
if not os.path.exists(input_dir):
    raise FileNotFoundError(f"Input directory '{input_dir}' does not exist.")
if not os.path.exists(annotations_dir):
    raise FileNotFoundError(f"Annotations directory '{annotations_dir}' does not exist.")

# Create output directories
os.makedirs(output_dir, exist_ok=True)
os.makedirs(augmented_annotations_dir, exist_ok=True)

## 🔄 Define Image Augmentation Function

This function applies several augmentations to an input image.

In [ ]:
def augment_image(image, angle=0, flip=False):
    if image.mode == "RGBA":
        image = image.convert("RGB")

    # Random rotation
    if angle != 0:
        image = image.rotate(angle, expand=True)

    # Random brightness adjustment
    if random.choice([True, False]):
        enhancer = ImageEnhance.Brightness(image)
        image = enhancer.enhance(random.uniform(0.7, 1.3))

    # Random color adjustment
    if random.choice([True, False]):
        enhancer = ImageEnhance.Color(image)
        image = enhancer.enhance(random.uniform(0.7, 1.3))

    # Random horizontal flip
    if flip:
        image = image.transpose(Image.FLIP_LEFT_RIGHT)

    return image

## 📝 Adjust Annotations for Augmented Images

This function updates the bounding box coordinates in the annotation files to match the applied image augmentations.

In [ ]:
# Adjust YOLO labels for flip and rotation
def augment_annotation(annotation_path, output_path, angle=0, flip=False, image_width=0, image_height=0):
    with open(annotation_path, 'r') as file:
        lines = file.readlines()

    augmented_lines = []
    for line in lines:
        parts = line.strip().split()
        class_id, x_center, y_center, width, height = map(float, parts)

        # Convert to absolute coordinates
        x_center_abs = x_center * image_width
        y_center_abs = y_center * image_height

        # Apply flip
        if flip:
            x_center_abs = image_width - x_center_abs

        # Apply rotation
        if angle != 0:
            angle_rad = math.radians(angle)
            x_new = math.cos(angle_rad) * (x_center_abs - image_width / 2) - math.sin(angle_rad) * (y_center_abs - image_height / 2) + image_width / 2
            y_new = math.sin(angle_rad) * (x_center_abs - image_width / 2) + math.cos(angle_rad) * (y_center_abs - image_height / 2) + image_height / 2
            x_center_abs, y_center_abs = x_new, y_new

        # Convert back to YOLO
        x_center = x_center_abs / image_width
        y_center = y_center_abs / image_height

        augmented_lines.append(f"{int(class_id)} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

    # Save
    with open(output_path, 'w') as file:
        file.writelines(augmented_lines)

## 🔄 Perform Augmentation on All Images

For each image in the dataset:
- Check if its annotation file exists; skip if missing.
- Generate 2 augmented versions with random horizontal flips (no rotation here).
- Save the augmented images and corresponding updated annotation files.
- Errors during processing are caught and reported without stopping the entire loop.

In [ ]:
image_files = [f for f in os.listdir(input_dir) if f.endswith(('.jpeg', '.jpg', '.png'))]
if not image_files:
    raise ValueError("No image files found.")

for img_file in image_files:
    try:
        img_path = os.path.join(input_dir, img_file)
        annotation_path = os.path.join(annotations_dir, os.path.splitext(img_file)[0] + ".txt")

        if not os.path.exists(annotation_path):
            print(f"Missing label for {img_file}. Skipping.")
            continue

        with Image.open(img_path) as img:
            width, height = img.size

            # Create 2 augmented versions
            for i in range(2):
                angle = random.randint(-15, 15)
                flip = random.choice([True, False])

                augmented_img = augment_image(img.copy(), angle=angle, flip=flip)

                out_img_name = f"aug_{i}_{img_file}"
                out_img_path = os.path.join(output_dir, out_img_name)
                augmented_img.save(out_img_path, format="JPEG")

                out_label_name = f"aug_{i}_{os.path.splitext(img_file)[0]}.txt"
                out_label_path = os.path.join(augmented_annotations_dir, out_label_name)

                augment_annotation(
                    annotation_path,
                    out_label_path,
                    angle=angle,
                    flip=flip,
                    image_width=width,
                    image_height=height
                )

    except Exception as e:
        print(f"Error processing {img_file}: {e}")

## 🧪 Function to Visualize Image and Its Augmentations

In [ ]:
# Draw bounding boxes on image
def draw_annotations(image, annotation_path):
    with open(annotation_path, 'r') as file:
        lines = file.readlines()

    draw = ImageDraw.Draw(image)
    width, height = image.size

    for line in lines:
        class_id, x_center, y_center, box_width, box_height = map(float, line.strip().split())
        x_min = (x_center - box_width / 2) * width
        y_min = (y_center - box_height / 2) * height
        x_max = (x_center + box_width / 2) * width
        y_max = (y_center + box_height / 2) * height

        color = "green" if class_id == 0 else "red"
        draw.rectangle([x_min, y_min, x_max, y_max], outline=color, width=2)
        draw.text((x_min, y_min), str(int(class_id)), fill=color)

    return image

## 📸 Visualize Original and Augmented Images with Annotations

- Select a sample image from the dataset.
- Display the original image with bounding boxes.
- Display augmented versions

In [ ]:
# Show original and augmented images
import matplotlib.pyplot as plt

sample_img = random.choice(image_files)
img_path = os.path.join(input_dir, sample_img)
ann_path = os.path.join(annotations_dir, os.path.splitext(sample_img)[0] + ".txt")

with Image.open(img_path) as img:
    width, height = img.size
    plt.figure(figsize=(15, 5))

    # Original
    annotated = draw_annotations(img.copy(), ann_path)
    plt.subplot(1, 3, 1)
    plt.imshow(annotated)
    plt.title("Original")
    plt.axis("off")

    # Augmented versions
    for i in range(2):
        aug_img_path = os.path.join(output_dir, f"aug_{i}_{sample_img}")
        aug_ann_path = os.path.join(augmented_annotations_dir, f"aug_{i}_{os.path.splitext(sample_img)[0]}.txt")

        if not os.path.exists(aug_img_path) or not os.path.exists(aug_ann_path):
            continue

        with Image.open(aug_img_path) as aug_img:
            annotated_aug = draw_annotations(aug_img.copy(), aug_ann_path)
            plt.subplot(1, 3, i + 2)
            plt.imshow(annotated_aug)
            plt.title(f"Augmentation {i+1}")
            plt.axis("off")

    plt.tight_layout()
    plt.show()

## ✅ Done with Augmentation!
### Your dataset is now augmented and ready for training.

### Continue to the next notebook to train a YOLOv11 model using your new dataset and evaluate it on test images.

#### 📦 **Next Step: [3) Model Training & Inference.ipynb](./3_Data_Augmentation.ipynb)**
---